In [9]:
import os
import sys
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
from dask.distributed import Client

/home/ubuntu/lsst_stack_26/conda/envs/lsst-scipipe-7.0.1/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


In [10]:
client = Client(n_workers=8)

2024-02-20 12:23:51,451 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 40450ed845c7519a30391c77c2710fd3 initialized by task ('shuffle-transfer-40450ed845c7519a30391c77c2710fd3', 796) executed on worker tcp://127.0.0.1:44575
2024-02-20 12:27:16,527 - distributed.shuffle._scheduler_plugin - WARNING - Shuffle 40450ed845c7519a30391c77c2710fd3 deactivated due to stimulus 'task-finished-1708432036.523974'


In [11]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 31.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45019,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 31.36 GiB
Comm: tcp://127.0.0.1:46161,Total threads: 1
Dashboard: http://127.0.0.1:40469/status,Memory: 3.92 GiB
Nanny: tcp://127.0.0.1:34647,


In [12]:
url_path = '/data/butler/dp02/dp02_products_datasets_urls.txt'

In [13]:
urls = dd.read_csv(url_path, header=None, dtype=str, names=['urls'])

In [14]:
urls.head()

,urls
0,https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...
1,https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
2,https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
3,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
4,https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...


In [16]:
# coadd_urls = urls['Coadd' in urls.urls.str]
coadd_urls = urls[urls['urls'].str.contains('Coadd', na=False)]

In [17]:
coadd_urls.persist()

<dask_expr.expr.DataFrame: expr=FromGraph(c6f7b87)>

In [19]:
print(coadd_urls.head())

                                                 urls
13  https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
15  https://ccnetlsst04.in2p3.fr:65105/2.2i/runs/F...
45  https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...
75  https://ccnetlsst02.in2p3.fr:65105/2.2i/runs/F...
76  https://ccnetlsst03.in2p3.fr:65105/2.2i/runs/F...


In [20]:
coadd_urls['urls'].to_csv('/data/butler/dp02/coadd_urls_2.csv',single_file=True,header_first_partition_only=True)

['/data/butler/dp02/coadd_urls_2.csv']

In [21]:
def extract_directory(url,start,end):
    #trying to see what directories are there
    directories=url.split('/')[start:end]
    return '/'.join(directories)

In [22]:
coadd_urls['dir']= coadd_urls['urls'].apply(lambda x: extract_directory(x,10,11), meta=('str'))

In [23]:
coadd_urls['dir'].head()

13    deepCoadd_psfMatchedWarp
15        deepCoadd_directWarp
45        deepCoadd_directWarp
75                   deepCoadd
76            deepCoadd_nImage
Name: dir, dtype: object

In [24]:
directory_counts = coadd_urls['dir'].value_counts().compute()

In [33]:
directory_counts

dir
deepCoadd_forced_src                                         46704
deepCoadd_calexp_background                                  46739
forcedPhotCoadd_log                                          46703
deepCoadd_deblendedFlux_schema                                  50
deepCoadd_deblendedFlux                                      46704
deepCoadd_nImage                                             46739
deepCoadd_dcr_ddec_consolidated_map_weighted_mean               11
deepCoadd_psf_maglim_consolidated_map_weighted_mean             12
forcedPhotCoadd_metadata                                     46704
goodSeeingCoadd_nImage                                       46739
deepCoadd_sky_noise_consolidated_map_weighted_mean              11
deepCoadd_meas_schema                                           54
deepCoadd_directWarp                                       4450147
deepCoadd_dcr_dra_map_weighted_mean                            952
deepCoadd_ref_schema                                      

In [30]:
directory_counts_2 = directory_counts.reset_index()

In [39]:
directory_counts_2

,dir,count
0,deepCoadd_forced_src,46704
1,deepCoadd_calexp_background,46739
2,forcedPhotCoadd_log,46703
3,deepCoadd_deblendedFlux_schema,50
4,deepCoadd_deblendedFlux,46704
5,deepCoadd_nImage,46739
6,deepCoadd_dcr_ddec_consolidated_map_weighted_mean,11
7,deepCoadd_psf_maglim_consolidated_map_weighted...,12
8,forcedPhotCoadd_metadata,46704
9,goodSeeingCoadd_nImage,46739


In [40]:
type(directory_counts_2)

pandas.core.frame.DataFrame

In [41]:
directory_counts_2.to_csv('directory_counts.csv',index=False)

In [42]:
import pandas as pd

In [70]:
coadds_us = pd.read_csv('coadd_ds_us.csv')

In [71]:
coadds_us['dir'] = coadds_us['dir'].astype(str)
coadds_us

,dir,storageClass
0,deepCoadd_calexp,ExposureF
1,deepCoadd_calexp_background,Background
2,deepCoadd_det,SourceCatalog
3,deepCoadd,ExposureF
4,deepCoadd_nImage,ImageU
...,...,...
79,injected_deepCoadd_obj,DataFrame
80,injected_deepCoadd_forced_src,SourceCatalog
81,injected_deepCoadd_measMatch,Catalog
82,injected_deepCoadd_measMatchFull,Catalog


In [68]:
directory_counts_2['dir'] = directory_counts_2['dir'].astype(str)
directory_counts_2

,dir,count
0,deepCoadd_forced_src,46704
1,deepCoadd_calexp_background,46739
2,forcedPhotCoadd_log,46703
3,deepCoadd_deblendedFlux_schema,50
4,deepCoadd_deblendedFlux,46704
5,deepCoadd_nImage,46739
6,deepCoadd_dcr_ddec_consolidated_map_weighted_mean,11
7,deepCoadd_psf_maglim_consolidated_map_weighted...,12
8,forcedPhotCoadd_metadata,46704
9,goodSeeingCoadd_nImage,46739


In [97]:
#dataset_types=directory_counts_2.merge(coadds_us,on='dir')
dataset_types=coadds_us.merge(directory_counts_2,on='dir')


In [98]:
dataset_types


,dir,storageClass,count
0,deepCoadd_calexp,ExposureF,46739
1,deepCoadd_calexp_background,Background,46739
2,deepCoadd_det,SourceCatalog,46739
3,deepCoadd,ExposureF,46739
4,deepCoadd_nImage,ImageU,46739
5,deepCoadd_ref_schema,SourceCatalog,54
6,deepCoadd_mergeDet_schema,SourceCatalog,50
7,deepCoadd_deblendedFlux_schema,SourceCatalog,50
8,deepCoadd_forced_src_schema,SourceCatalog,57
9,deepCoadd_meas_schema,SourceCatalog,54


In [86]:
catalogues = dataset_types[dataset_types[' storageClass'].str.contains('Catalog')]

In [87]:
catalogues

,dir,count,storageClass
0,deepCoadd_forced_src,46704,SourceCatalog
3,deepCoadd_deblendedFlux_schema,50,SourceCatalog
4,deepCoadd_deblendedFlux,46704,SourceCatalog
11,deepCoadd_meas_schema,54,SourceCatalog
14,deepCoadd_ref_schema,54,SourceCatalog
16,deepCoadd_ref,7784,SourceCatalog
28,deepCoadd_mergeDet,7784,SourceCatalog
29,deepCoadd_peak_schema,50,PeakCatalog
34,deepCoadd_forced_src_schema,57,SourceCatalog
36,deepCoadd_measMatchFull,46704,Catalog


In [90]:
a = [32,0,28,0,0,5,0,0,0,0,0,0,0,32]

In [93]:
catalogues.loc[:,'TB'] = a

In [94]:
catalogues

,dir,count,storageClass,TB
0,deepCoadd_forced_src,46704,SourceCatalog,32
3,deepCoadd_deblendedFlux_schema,50,SourceCatalog,0
4,deepCoadd_deblendedFlux,46704,SourceCatalog,28
11,deepCoadd_meas_schema,54,SourceCatalog,0
14,deepCoadd_ref_schema,54,SourceCatalog,0
16,deepCoadd_ref,7784,SourceCatalog,5
28,deepCoadd_mergeDet,7784,SourceCatalog,0
29,deepCoadd_peak_schema,50,PeakCatalog,0
34,deepCoadd_forced_src_schema,57,SourceCatalog,0
36,deepCoadd_measMatchFull,46704,Catalog,0


In [95]:
catalogues.TB.sum()

97

In [99]:
catalogues[['dir']]

,dir
0,deepCoadd_forced_src
3,deepCoadd_deblendedFlux_schema
4,deepCoadd_deblendedFlux
11,deepCoadd_meas_schema
14,deepCoadd_ref_schema
16,deepCoadd_ref
28,deepCoadd_mergeDet
29,deepCoadd_peak_schema
34,deepCoadd_forced_src_schema
36,deepCoadd_measMatchFull


Process Dask Worker process (from Nanny):
2024-02-20 14:32:21,562 - distributed.nanny - ERROR - Worker process died unexpectedly
2024-02-20 14:32:21,562 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-02-20 14:32:21,562 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2024-02-20 14:32:21,563 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2024-02-20 14:32:21,565 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rec

In [100]:
dataset_types.to_csv('coadd_dataset_types.csv',index=False)

In [101]:
client.close()